In [33]:
%load_ext autoreload

%autoreload 2
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score, roc_curve

from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import LSTM


from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence


import warnings
import numpy as np
from collections import OrderedDict

import os

from lob_data_utils import lob, db_result, gdf_pca, model
from lob_data_utils.svm_calculation import lob_svm


sns.set_style('whitegrid')
warnings.filterwarnings('ignore')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
data_length = 15000
stock = '9061'
gdf_filename_pattern = 'gdf_{}_' + 'len{}'.format(data_length) + '_r{}_s{}_K50'
gdf_parameters = [(1.0, 1.0), (0.1, 0.1), (0.1, 1.0), (1.0, 0.1), (0.01, 0.1)]

In [3]:
gdf_dfs = []
for r, s in gdf_parameters:
    gdf_dfs.append(gdf_pca.SvmGdfResults(
        stock,  r=r, s=s, data_length=data_length, 
        gdf_filename_pattern=gdf_filename_pattern))

In [11]:
gdf_dfs[1].df.columns

Index(['Unnamed: 0', 'gdf_0', 'gdf_1', 'gdf_10', 'gdf_11', 'gdf_12', 'gdf_13',
       'gdf_14', 'gdf_15', 'gdf_16', 'gdf_17', 'gdf_18', 'gdf_19', 'gdf_2',
       'gdf_20', 'gdf_21', 'gdf_22', 'gdf_23', 'gdf_24', 'gdf_25', 'gdf_26',
       'gdf_27', 'gdf_28', 'gdf_29', 'gdf_3', 'gdf_30', 'gdf_31', 'gdf_32',
       'gdf_33', 'gdf_34', 'gdf_35', 'gdf_36', 'gdf_37', 'gdf_38', 'gdf_39',
       'gdf_4', 'gdf_40', 'gdf_41', 'gdf_42', 'gdf_43', 'gdf_44', 'gdf_45',
       'gdf_46', 'gdf_47', 'gdf_48', 'gdf_49', 'gdf_5', 'gdf_6', 'gdf_7',
       'gdf_8', 'gdf_9', 'mid_price', 'mid_price_indicator', 'datetime',
       'bid_price', 'ask_price', 'queue_imbalance', 'prev_queue_imbalance'],
      dtype='object')

In [30]:
def as_keras_metric(method):
    import functools
    from keras import backend as K
    import tensorflow as tf
    @functools.wraps(method)
    def wrapper(self, args, **kwargs):
        """ Wrapper for turning tensorflow metrics into keras metrics """
        value, update_op = method(self, args, **kwargs)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([update_op]):
            value = tf.identity(value)
        return value
    return wrapper
import tensorflow as tf
auc_roc = as_keras_metric(tf.metrics.auc)

In [48]:
X_train = gdf_dfs[4].df[[gdf for gdf in gdf_dfs[0].df.columns if 'gdf' in gdf or 'queue' in gdf]].values

pca = PCA(n_components=10)
pca.fit(X_train)
X_train = pca.transform(X_train)

print(X_train.shape)
X_train = np.reshape(X_train, (X_train.shape[0], 1, 10))
y_train = gdf_dfs[4].df['mid_price_indicator'].values
#y_train = np.reshape(y_train, (y_train.shape[0], 1, y_train.shape[0]))

(11999, 10)


In [60]:
max_features = 1024
model = Sequential()
model.add(LSTM(128))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[auc_roc])

model.fit(X_train, y_train, epochs=3, batch_size=10)
print(model.summary())

Epoch 1/3
11999/11999 [==============================] - 22s 2ms/step - loss: 0.6872 - auc: 0.5495
Epoch 2/3
11999/11999 [==============================] - 18s 2ms/step - loss: 0.6864 - auc: 0.5648
Epoch 3/3
11999/11999 [==============================] - 19s 2ms/step - loss: 0.6862 - auc: 0.5645
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_25 (LSTM)               (None, 128)               71168     
_________________________________________________________________
dense_36 (Dense)             (None, 1)                 129       
Total params: 71,297
Trainable params: 71,297
Non-trainable params: 0
_________________________________________________________________
None


In [61]:
X_test = gdf_dfs[4].df_test[[gdf for gdf in gdf_dfs[0].df_test.columns if 'gdf' in gdf or 'queue' in gdf]].values

X_test = pca.transform(X_test)


print(X_test.shape)
X_test= np.reshape(X_test, (X_test.shape[0], 1, 10))
y_test = gdf_dfs[4].df_test['mid_price_indicator'].values

(2999, 10)


In [62]:
pred_test = model.predict(X_test)

pred = model.predict(X_train)
from sklearn import metrics
metrics.roc_auc_score(y_test, pred_test), metrics.roc_auc_score(y_train, pred)

(0.5420854148431218, 0.5697937396769125)